# Register MLFlow Model

- Download notebook https://github.com/MicrosoftLearning/mslearn-azure-ml/blob/main/Labs/10/Log%20models%20with%20MLflow.ipynb
(clone whole repo from https://github.com/MicrosoftLearning/mslearn-azure-ml.git)

- In notebook, replace 
```
compute="aml-cluster",
```

with your compute instance's name.

## Set up

In [1]:
# AML imports
from azure.ai.ml import (
    command,
    MLClient
)
from azure.identity import DefaultAzureCredential

# authenticate
credential = DefaultAzureCredential()

# Get a handle to the workspace
ml_client = MLClient.from_config (
    credential=credential
)

Found the config file in: /config.json


## Summary:

#### 1. Flavor

Both

```python
mlflow.sklearn.autolog()
```

and 

```python
mlflow.autolog()
```

achieve the same

#### 2. Separate model logging

We can avoid logging the model and do it later, by doing:

```python
mlflow.autolog(log_models=False)
```

and later doing:

##### 2.1. Inferred signature

If we want the signature to be inferred we run:

```python

from mlflow.models.signature import infer_signature

# create the signature by inferring it from the datasets
signature = infer_signature(X_train, y_hat)

# manually log the model
mlflow.sklearn.log_model(model, "model", signature=signature)    
```

##### 2.2. Manually indicating signature

If we want to indicate it manually, we would do something like:

```python
# create the signature manually
input_schema = Schema([
    ColSpec("integer", "Pregnancies"),
    ColSpec("integer", "PlasmaGlucose"),
    ColSpec("integer", "DiastolicBloodPressure"),
    ColSpec("integer", "TricepsThickness"),
    ColSpec("integer", "DiastolicBloodPressure"),
    ColSpec("integer", "SerumInsulin"),
    ColSpec("double", "BMI"),
    ColSpec("double", "DiabetesPedigree"),
    ColSpec("integer", "Age"),
])

output_schema = Schema([ColSpec("boolean")])

# Create the signature object
signature = ModelSignature(inputs=input_schema, outputs=output_schema)

# manually log the model
mlflow.sklearn.log_model(model, "model", signature=signature)

```

### Register model:

```python
from azure.ai.ml.entities import Model
from azure.ai.ml.constants import AssetTypes

job_name = returned_job.name

run_model = Model(
    path=f"azureml://jobs/{job_name}/outputs/artifacts/paths/model/",
    name="mlflow-diabetes",
    description="Model created from run.",
    type=AssetTypes.MLFLOW_MODEL,
)
# Uncomment after adding required details above
ml_client.models.create_or_update(run_model)
```

In the Studio, navigate to the **Models** page. In the model list, find the `mlflow-diabetes` model and select it to explore it further.

- In the **Details** tab of the `mlflow-diabetes` model, you can review that it's a `MLFLOW` type model and the job that trained the model.
- In the **Artifacts** tab you can find the directory with the `MLmodel` file.

If you want to explore the model's behavior further, you can **optionally** choose to deploy the model to a real-time endpoint.

### Metrics:

- We see png plots in the outputs+logs tab and in the images tab. These are logged automatically with autolog, during training.
- For the moment, I don't see the plots manually saved in the code, in the `train-model-sklearn.py` script (second one in the notebook)